## H20 automl: freMTPL2freq (Regression)

Based on: https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_regression_powerplant_output.ipynb

Same as "automl_freMTPL2freq.ipynb" except for stopping rule.

Tried to use Poisson distribution, but that failed (despite examples showing how to do that - examples failed too).

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "MAE"  # specify your evaluation metric
stopping = "MAE"  # specify your stopping metric
target = "ClaimNb" # name of target or label variable

## Import

In [2]:
# Import packages

from h2o.automl import H2OAutoML

import h2o

In [3]:
h2o.__version__

'3.38.0.1'

In [4]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_372"; OpenJDK Runtime Environment (build 1.8.0_372-b07); OpenJDK 64-Bit Server VM (build 25.372-b07, mixed mode)
  Starting server from /home/stever7/.local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpi1sgevaq
  JVM stdout: /tmp/tmpi1sgevaq/h2o_stever7_started_from_python.out
  JVM stderr: /tmp/tmpi1sgevaq/h2o_stever7_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,10 months and 19 days !!!
H2O_cluster_name:,H2O_from_python_stever7_36xi42
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.62 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


In [5]:
# Import data

train = h2o.import_file("freMTPL2freq_dataset_train.csv")
test = h2o.import_file("freMTPL2freq_dataset_test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(474765, 12)

In [7]:
test.shape

(203226, 12)

In [8]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,10 months and 19 days !!!
H2O_cluster_name:,H2O_from_python_stever7_36xi42
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.62 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


## Tidy

(skipped)

## Transform

In [9]:
predictors = train.columns
predictors

['IDpol',
 'ClaimNb',
 'Exposure',
 'VehPower',
 'VehAge',
 'DrivAge',
 'BonusMalus',
 'VehBrand',
 'VehGas',
 'Area',
 'Density',
 'Region']

In [10]:
predictors.remove(target)  # remove target variable
predictors.remove("IDpol")  # remove "IDpol" (should not be a predictor)

## Visualize

(skipped)

## Model

In [11]:
%%time

aml = H2OAutoML(
    max_runtime_secs=time_limit, 
    seed=1, 
    sort_metric=metric,
    stopping_metric=stopping,
    distribution="poisson"
    # distribution=dict(type="tweedie", tweedie_power=1.0)
)

CPU times: user 4.24 ms, sys: 229 µs, total: 4.47 ms
Wall time: 43.6 ms


In [12]:
%%time

aml.train(x=predictors, y=target, training_frame=train, leaderboard_frame=test)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

19:40:35.233: GBM_lr_annealing_selection_AutoML_1_20230807_184045 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

CPU times: user 26.6 s, sys: 6.79 s, total: 33.4 s
Wall time: 1h 3s


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_1_20230807_184045_model_1


Status of Neuron Layers: predicting ClaimNb, regression, poisson distribution, Automatic loss, 15,401 weights/biases, 188.8 KB, 98,120 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ---------------------  -----------------------
    1        51       Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.08870884375963428    0.26178646087646484   0.0         -0.0012683652447070498  0.11474719643592834  0.5009849322423381     0.10765299201011658
    3        100      RectifierDropout  10.0       0.0   0.0   0.018461870710580842   0.033664703369140625  0.0         0.00192289263941733     0.10671007633209229  0.9899262288121728     0.0975622832775116
    4        1        Linear                       0.0   0.0   0.0026364072487922384  0.007698049768805504  0.0         0.0024498701374977825   0.115377277135849    -0.004328970883303774  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.04142625617110249
RMSE: 0.20353441028755428
MAE: 0.04797508382147716
RMSLE: 0.13559922065703245
Mean Residual Deviance: 0.3720107140433803

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.04323060988116899
RMSE: 0.20791971979869775
MAE: 0.042512150514254034
RMSLE: 0.13696455539929356
Mean Residual Deviance: 0.4559442203377824

Cross-Validation Metrics Summary: 
                        mean        sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     0.0432687   0.00197383   0.0399002     0.0442859     0.0449345     0.0439215     0.0433012
mean_residual_deviance  0.445961    0.0862316    0.593903      0.38131       0.414491      0.445975      0.394128
mse                     0.0431582   0.000981591  0.0436895     0.0428604     0.0424709     0.0445908     0.0421793
r2                      -0.0223914  0.00701023   -0.034423     -0.0162358    -0.0204029    -0.021546     -0.0193494
residual_deviance       0.445961    0.0862316    0.593903      0.38131       0.414491      0.445975      0.394128
rmse                    0.207735    0.00235633   0.20902       0.207027      0.206085      0.211165      0.205376
rmsle                   0.136798    0.00184491   0.139046      0.134737      0.13738       0.13777       0.135058

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-08-07 19:38:45  0.000 sec                           0         0             0          nan              nan                  nan             nan
    2023-08-07 19:38:47  1 min  8.812 sec  57346 obs/sec     0.206671  1             98120      0.203534         0.372011             0.0479751       -0.014887

Variable Importances: 
variable                     relative_importance    scaled_importance    percentage
---------------------------  ---------------------  -------------------  --------------------
Region.Bourgogne             1.0                    1.0                  0.02390629049706352
DrivAge                      0.9923180937767029     0.9923180937767029   0.023722644615318176
Region.Corse                 0.9847779870033264    

In [13]:
aml.leaderboard.head(rows=aml.leaderboard.nrows)

model_id,mae,rmse,mse,rmsle,mean_residual_deviance
DeepLearning_grid_2_AutoML_1_20230807_184045_model_1,0.0410376,0.214523,0.0460203,0.140216,0.556712
GBM_grid_1_AutoML_1_20230807_184045_model_41,0.0424882,0.221542,0.0490808,0.142885,1.42153
GBM_grid_1_AutoML_1_20230807_184045_model_68,0.0434099,0.216318,0.0467933,0.142321,1.21797
DeepLearning_grid_1_AutoML_1_20230807_184045_model_4,0.0441542,0.213744,0.0456863,0.139448,0.433797
DeepLearning_grid_1_AutoML_1_20230807_184045_model_1,0.0442469,0.213436,0.0455551,0.139204,0.50357
DeepLearning_grid_1_AutoML_1_20230807_184045_model_3,0.0450324,0.213222,0.0454636,0.139007,0.483823
GBM_grid_1_AutoML_1_20230807_184045_model_5,0.0452073,0.220798,0.0487519,0.144382,1.12788
GBM_grid_1_AutoML_1_20230807_184045_model_13,0.046664,0.223585,0.0499901,0.144032,1.03248
GBM_grid_1_AutoML_1_20230807_184045_model_43,0.0471181,0.240949,0.0580566,0.145065,0.994786
GBM_grid_1_AutoML_1_20230807_184045_model_58,0.0489051,0.219652,0.0482471,0.143371,0.993581


In [14]:
## Predict Using Leader Model
pred = aml.predict(test)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [15]:
perf = aml.leader.model_performance(test)
perf

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 0.04602026996959205
RMSE: 0.2145233553009836
MAE: 0.041037551419353675
RMSLE: 0.14021575629154057
Mean Residual Deviance: 0.55671202024049

## Communicate

(skipped)